In [2]:
import os
import jieba
import warnings
warnings.filterwarnings('ignore')

In [3]:
novel_path = "金庸小说精校版/"
data_path = "./"

In [4]:
# 加载停用词表
stop_words_file = open(data_path + "stop_words.txt", 'r',encoding="gbk",)
stop_words = list()
for line in stop_words_file.readlines():
    line = line.strip()   # 去掉每行末尾的换行符
    stop_words.append(line)
stop_words_file.close()
print(len(stop_words))
print(stop_words[300:320])

1903
['乘势', '乘机', '乘胜', '乘虚', '乘隙', '九', '也', '也好', '也就是说', '也是', '也罢', '了', '了解', '争取', '二', '二来', '二话不说', '二话没说', '于', '于是']


分词前在词库中加入人物名称、武功名称、门派名称

In [5]:
# 导入金庸小说人物
people_names_file = open(data_path + "金庸小说全人物.txt", 'r',encoding="gbk",)
people_names = list()
for line in people_names_file.readlines():
    line = line.strip()   # 去掉每行末尾的换行符
    jieba.add_word(line)
    people_names.append(line)
stop_words_file.close()
print(len(people_names))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d9/1hp8c1n16xq2gypxpzgt_9m40000gn/T/jieba.cache
Loading model cost 0.547 seconds.
Prefix dict has been built successfully.


1237


In [6]:
# 导入金庸小说武功
kungfu_names_file = open(data_path + "金庸小说全武功.txt", 'r',encoding="gbk",)
kungfu_names = list()
for line in kungfu_names_file.readlines():
    line = line.strip()   # 去掉每行末尾的换行符
    jieba.add_word(line)
    kungfu_names.append(line)
stop_words_file.close()
print(len(kungfu_names))

389


In [7]:
# 导入金庸小说门派
sect_names_file = open(data_path + "金庸小说全门派.txt", 'r',encoding="gbk",)
sect_names = list()
for line in sect_names_file.readlines():
    line = line.strip()   # 去掉每行末尾的换行符
    jieba.add_word(line)
    sect_names.append(line)
stop_words_file.close()
print(len(sect_names))

97


In [8]:
novel_names = list(os.listdir(novel_path))
novel_names = [i for i in novel_names if i.endswith(".txt")]

In [9]:
novel_names 

['侠客行.txt',
 '射雕英雄传.txt',
 '鹿鼎记.txt',
 '越女剑.txt',
 '雪山飞狐.txt',
 '连城诀.txt',
 '飞狐外传.txt',
 '神雕侠侣.txt',
 '鸳鸯刀.txt',
 '天龙八部.txt',
 '碧血剑.txt',
 '白马啸西风.txt',
 '笑傲江湖.txt',
 '倚天屠龙记.txt',
 '书剑恩仇录.txt']

In [12]:
from multiprocessing import Pool,Queue
import time

class test:
    def __init__(self, novel_names):
        self.a = novel_names
    def run(self):
        in_queue, out_queue = Queue(), Queue()
        for i in self.a:
            in_queue.put(i)
        def f(in_queue, out_queue):
            while not in_queue.empty():
                novel_name = in_queue.get()
                tmp = []
                novel = open(novel_path + novel_name, 'r', encoding='utf-8-sig')
                print("Waiting for {}...".format(novel_name))
                line = novel.readline()
                while line:
                    line_1 = line.strip()
                    outstr = ''
                    line_seg = jieba.cut(line_1, cut_all=False)
                    for word in line_seg:  
                        if word not in stop_words:
                            if word != '\t':
                                if word[:2] in people_names:
                                    word = word[:2]
                                outstr += word 
                                outstr += " "
                    if len(str(outstr.strip())) != 0:
                        tmp.append(str(outstr.strip()).split())
                    line = novel.readline()
                print("{} finished，with {} Row".format(novel_name, (len(tmp))))
                out_queue.put(tmp)
        pool = Pool(5, f, (in_queue, out_queue))
        self.b = []
        while len(self.b) < len(self.a):
            if not out_queue.empty():
                t = out_queue.get()
                self.b.extend(t)
        pool.terminate()
        
t = test(novel_names)
t.run()

Waiting for 侠客行.txt...
Waiting for 射雕英雄传.txt...
Waiting for 鹿鼎记.txt...
Waiting for 越女剑.txt...
Waiting for 雪山飞狐.txt...
越女剑.txt finished，with 197 Row
Waiting for 连城诀.txt...


In [11]:
print(len(t.b))

15


In [19]:
# print(t.b[0])

In [9]:
seg_novel = []
for novel_name in novel_names:
    novel = open(novel_path + novel_name, 'r', encoding='utf-8-sig')
    print("Waiting for {}...".format(novel_name))
    line = novel.readline()
    forward_rows = len(seg_novel)
    while line:
        line_1 = line.strip()
        outstr = ''
        line_seg = jieba.cut(line_1, cut_all=False)
        for word in line_seg:  
            if word not in stop_words:
                if word != '\t':
                    if word[:2] in people_names:
                        word = word[:2]
                    outstr += word 
                    outstr += " "
        if len(str(outstr.strip())) != 0:
            seg_novel.append(str(outstr.strip()).split())
        line = novel.readline()
    print("{} finished，with {} Row".format(novel_name, (len(seg_novel) - forward_rows)))
    print("-" * 40)
print("-" * 40)
print("-" * 40)
print("All finished，with {} Row".format(len(seg_novel)))

Waiting for 侠客行.txt...
侠客行.txt finished，with 3514 Row
----------------------------------------
Waiting for 射雕英雄传.txt...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/coming/opt/anaconda3/envs/tf21/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e31290842414>", line 14, in <module>
    if word[:2] in people_names:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/coming/opt/anaconda3/envs/tf21/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/coming/opt/anaconda3/envs/tf21/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset

TypeError: object of type 'NoneType' has no len()

In [25]:
for line in seg_novel[30000:30010]:
    print(line)

['东南', '形胜', '吴', '钱塘', '自古', '繁华', '烟柳', '画桥', '风帘翠', '幙', '参差', '十万', '云树绕', '堤沙', '怒涛', '卷', '霜雪', '天堑', '无涯', '市列', '珠玑', '户盈', '罗绮', '竞', '豪奢', '重湖', '叠', '山献', '清佳', '三秋', '桂子', '十里', '荷花', '羌管', '弄', '晴', '菱歌', '泛夜', '嬉嬉', '钓叟莲娃', '千骑', '拥高牙', '乘醉', '听', '箫', '鼓', '吟赏', '烟霞', '异日图', '好景', '归去', '凤池', '夸']
['郭靖', '懂', '咿', '咿', '地唱些', '但觉', '牙板', '轻击', '箫声', '悠扬', '甚', '动听', '一曲', '已毕', '完颜洪烈', '杨康', '齐声', '赞道', '唱得', '歌妓', '道谢', '喜气洋洋', '乐师', '想', '完颜洪烈', '赏得']
['听', '完颜洪烈', '孩儿', '柳永', '一首', '海潮', '词', '大金国', '一段', '因缘', '杨康', '孩儿', '不知', '请', '爹爹']
['郭靖', '黄蓉', '完颜洪烈', '做', '爹爹', '语气', '间', '亲热', '相互', '一眼', '郭靖', '气恼', '难受', '恨不得', '揪住', '问个', '明白']
['听', '完颜洪烈', '我大金', '正隆', '年间', '我大', '金主', '金主亮', '见到', '柳永', '这首词', '西湖', '风景', '欣然', '有慕', '派遣', '使者', '南下', '时', '派', '著名', '画工', '摹写', '一幅', '临安', '城', '山水', '图画', '金主', '状貌', '策马', '立在', '临安', '城内', '吴山之', '顶', '金主', '画', '题诗', '万里', '车书', '混同', '江南', '别疆封', '提兵', '百万', '西湖', '吴山', '第一峰', '杨康', '赞道', '豪壮', '气概', '郭靖', '恼怒',

In [26]:
import gensim.models as w2v

In [27]:
model = w2v.Word2Vec(sentences=seg_novel, size=200, window=5, min_count=5, workers=4, sg=1)

In [28]:
# 寻找相似关系
def find_relation(a, b, c):
    d, _ = model.most_similar(positive=[c, b], negative=[a])[0]
    print (c,d)

In [38]:
find_relation("武当派","张三丰","天地会")

天地会 陈近南


In [30]:
model.most_similar("张无忌", topn=10)

[('赵敏', 0.5751789212226868),
 ('周芷若', 0.537672221660614),
 ('杨逍', 0.5256727337837219),
 ('小昭', 0.5213794112205505),
 ('东方不败', 0.4729505479335785),
 ('波斯', 0.46848881244659424),
 ('范遥', 0.4617539048194885),
 ('韦一笑', 0.4586241841316223),
 ('任我行', 0.4548007845878601),
 ('明教', 0.45135754346847534)]

In [31]:
model.save(data_path + 'all_skip_gram.model')

In [32]:
model = w2v.Word2Vec.load(data_path + 'all_skip_gram.model')

In [33]:
print(model.similarity('张无忌', '周芷若'))
print(model.similarity('张无忌', '赵敏'))

0.53767216
0.575179


In [34]:
print(model.most_similar("韦小宝", topn=10))

[('多隆', 0.558411180973053), ('索额图', 0.5404623746871948), ('康熙', 0.5350710153579712), ('小桂子', 0.528827428817749), ('建宁公主', 0.5188446640968323), ('奴才', 0.5129232406616211), ('施琅', 0.511640727519989), ('双儿', 0.5040061473846436), ('太监', 0.4895724058151245), ('苏菲亚', 0.48741617798805237)]


In [35]:
model.most_similar("王重阳", topn=10)

[('宝典', 0.8510153293609619),
 ('神通', 0.8482961654663086),
 ('窥', 0.8437038660049438),
 ('研习', 0.8386595249176025),
 ('所记', 0.8381162881851196),
 ('先天功', 0.8362185955047607),
 ('功诀', 0.8350870609283447),
 ('参悟', 0.8318731784820557),
 ('华山论剑', 0.8313177824020386),
 ('林朝英', 0.8311183452606201)]

In [36]:
print(model.similarity('韦小宝', '阿珂'))
print(model.similarity('韦小宝', '双儿'))
print(model.similarity('韦小宝', '建宁公主'))
print(model.similarity('韦小宝', '苏荃'))
print(model.similarity('韦小宝', '沐剑屏'))
print(model.similarity('韦小宝', '曾柔'))
print(model.similarity('韦小宝', '方怡'))

0.34928897
0.50400615
0.51884466
0.45665407
0.36300948
0.3843639
0.41635066


In [37]:
find_relation("杨过","小龙女","乔峰")

乔峰 阿朱
